This notebook is for querying the dataset and preprocessing the results

Expected input: None

Expected output: Cleaned dataset ready for NER or classification

In [1]:
import utils as ut
import pandas as pd
from pandas.api.types import is_numeric_dtype
import os
import hjson as json
import numpy as np
import string

import importlib
importlib.reload(ut)

/home/shared/shared_sandbox/lib/python3.8/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(
2023-02-22 05:28:50.444829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 05:28:51.070556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-22 05:28:51.070623: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

<module 'utils' from '/home/shared/code/08_protein_attribution/utils.py'>

In [2]:
# Read in our params file
f = open('input_params.hjson')
params = json.load(f)
f.close()

# SQL params
# db_type = params['core']['db_type']
# sql_code = params['core']['sql_code']

# Modelling params
itemname_col = params['core']['itemname_col']
spend_col = params['core']['spend_col']
cls_input_cols = params['core']['cls_input_cols']

ner_model_name = params['core']['ner_model_name']
cls_model_name = params['core']['cls_model_name']
model_type = 'classifier'
ner_model_type = 'named_entity_recognition'
sample_n = params['nb_three']['sample_n']
test_samples = params['nb_three']['test_samples']
n_strata = params['nb_three']['n_strata']
ner_clust_cols = params['core']['ner_clust_cols']

# # Algorithm specific params
use_pretrained_model = params['core']['use_pretrained_model']
model_architecture = params['core']['model_architecture']
model_path = params['core']['model_path']


In [3]:
db_type = 'redshift'

# The query must have the following: (a) an item name (b) a spend column
sql_code = '''
            select od.lineitem_name lineitem_name,
            tier.cleansed_tier_1 tier_1,
            tier.cleansed_tier_2 tier_2,
            tier.cleansed_tier_3 tier_3,
            tier.cleansed_tier_4 tier_4,
            sum(od.lineitem_price) as sales_amt_gross
            from "cdl_bi_edw"."dbt_dev"."orders" o
            left join "cdl_bi_edw"."dbt_dev"."order_details" od on o.key = od.order_fk
            left join "cdl_bi_edw"."datascience_item_dim"."consolidated_item_attributes_dbt" tier on od.lineitem_name = tier.item_name
            where 
                o.order_fiscal_date_fk between '2019-01-01' and '2022-12-31'
                and o.order_origin in ('pos', 'kiosk', 'mobile', 'web')
                and o.transaction_type in ('Sale', 'Unknown', 'Other')
                and o.check_outlier = 'No'
                and tier2 in ('Breakfast', 'Entree')
            group by lineitem_name, tier_1, tier_2, tier_3, tier_4
         '''

In [4]:
def preprocess_item_name(df, itemname_col):
    '''
    Add in any preprocessing your dataset needs in this step
    
    Other notes:
    item_input will be used for inference
    item_for_selection will be used for train and test set selection

    '''

    # # item_input is used for inference, for now we'll feed the model the input exactly as it's given to us.
    # # You can do some processing to item_input if you find that it breaks the model.
    # df['item_input'] = df[itemname_col]

    # item_for_selection is setting up a groupby later on which is looking to group variations of the same item.
    # Remove any special characters present in your item name here.
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) # Replace punctuation with ' '
    df['item_for_selection']= df[itemname_col].fillna('').str.translate(translator).str.lower().str.strip()

    #### Add in any preprocessing required here #####

    return df

def clean_column_names(df):
    # Clean up our column names to replace spaces with underscores and remove caps
    cols = list(df)
    cols_new = []

    for c in cols:
        c = c.lower().strip()
        c = c.replace(" ", "_")
        cols_new.append(c)

    df.columns = cols_new
    return df

def create_folders_for_project(model_type, cls_model_name, ner_model_name):
    # Create folders for this project if they don't exist

    if not os.path.exists(f'{model_type}/{cls_model_name}/data'):
        os.makedirs(f'{model_type}/{cls_model_name}/data')

    if not os.path.exists(f'{model_type}/{cls_model_name}/models'):
        os.makedirs(f'{model_type}/{cls_model_name}/models')

    if not os.path.exists(f'{ner_model_type}/{ner_model_name}/data'):
        os.makedirs(f'{ner_model_type}/{ner_model_name}/data')

    if not os.path.exists(f'{ner_model_type}/{ner_model_name}/models'):
        os.makedirs(f'{ner_model_type}/{ner_model_name}/models')

def query_data(db_type, sql_code):
    # Get the input data we'll use in this analysis

    query_file_name = db_type+'_query_results.csv'
    if os.path.isfile(query_file_name):
        df = pd.read_csv(query_file_name)
    else:
        query = sql_code
        if db_type == 'redshift':
            conn,cur = ut.redshift_conn()
        elif db_type == 'snowflake':
            conn,cur = ut.snowflake_conn()
        df = ut.psql_table_query_df(cur,query,ucase_cols = False)
        df.to_csv(query_file_name, index=False)
        conn.close()
    return df

def generate_clust_input(ner_model_name, itemname_col, ner_clust_cols, df):
    # Generate an input for our k-means clustering

    # This code should be moved to nb2 or 3
    # # If you ran notebooks 01 and 02, let's leverage the results from there. 
    # if os.path.isfile(f'named_entity_recognition/{ner_model_name}/data/{ner_model_name}_round1results.csv'):
    #     ner_df = pd.read_csv(f'named_entity_recognition/{ner_model_name}/data/{ner_model_name}_round1results.csv')
    #     ner_df = ner_df[[itemname_col] + ner_clust_cols].drop_duplicates()
    #     df = pd.merge(df,ner_df,how = 'left')
    #     df['clust_input'] = df[ner_clust_cols].fillna('').apply(lambda x: ' '.join(x), axis = 1).str.strip()
        
    # Let's use the full item name to cluster on
    df['clust_input'] = df['item_for_selection']
    return df

create_folders_for_project(model_type, cls_model_name, ner_model_name)
df = query_data(db_type, sql_code)
df = df.loc[df['tier_2'].isin(['Breakfast', 'Entree'])] # This can be removed the next time the query is run
df = clean_column_names(df)
df = preprocess_item_name(df, itemname_col)
df = generate_clust_input(ner_model_name, itemname_col, ner_clust_cols, df)



In [ ]:
df.to_csv(f'{model_type}/{cls_model_name}/data/{cls_model_name}_preprocessed.csv',index = False)
df.to_csv(f'{ner_model_type}/{ner_model_name}/data/{ner_model_name}_preprocessed.csv',index = False)

### Validation tests

In [ ]:
# Make sure our output df has the right columns
# If this fails, make sure that the columns tested for actually reside in your dataset

cols = list(df)
assert(all(item in cols for item in [itemname_col, spend_col, 'item_for_selection']))

In [ ]:
# Make sure our output df has at least one row
# If this fails, make sure your data wasn't accidentally dropped

assert(df.shape[0]>0)